In [1]:
import numpy as np
import cv2
import os
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from skimage.feature import hog
from skimage import exposure

In [81]:
def hog_features(image):
    image = cv2.resize(image, (64, 64))
    features = hog(image, pixels_per_cell=(4, 4), cells_per_block=(2, 2), transform_sqrt=True, block_norm="L2")
    return features

def get_data():
    features_lst, labels_lst = [], []
    im_dir = os.listdir('bbox')
    # for img_dir in im_dir:
    #     labels_lst.append('traffic-sign')
    #     img = cv2.imread(os.path.join('bbox', img_dir), cv2.IMREAD_GRAYSCALE)
    #     features_lst.append(hog_features(img))
    for img_dir in im_dir:
        number = int(img_dir.rsplit('_', 2)[-1].split('.')[0])
        labels_lst.append(number)
        img = cv2.imread(os.path.join('bbox', img_dir), cv2.IMREAD_GRAYSCALE)
        features_lst.append(hog_features(img))
    print("Read ", len(features_lst), " images")
    
    im_dir = os.listdir('bbox2')
    for img_dir in im_dir:
        labels_lst.append('background')
        img = cv2.imread(os.path.join('bbox2', img_dir), cv2.IMREAD_GRAYSCALE)
        features_lst.append(hog_features(img))
    print("Read ", len(features_lst), " images")
    return features_lst, labels_lst
# def get_data():
#     features_lst, labels_lst = [], []
#     im_dir = os.listdir('bbox')
#     for img_dir in im_dir:
#         number = int(img_dir.rsplit('_', 2)[-1].split('.')[0])
#         labels_lst.append(number)
#         img = cv2.imread(os.path.join('bbox', img_dir), cv2.IMREAD_GRAYSCALE)
#         features_lst.append(hog_features(img))
#     print("Read ", len(features_lst), " images")
#     return features_lst, labels_lst

In [82]:
features_lst, labels_lst = get_data()
classes_lst = list(set(labels_lst))

In [71]:
X_train, X_test, y_train, y_test = train_test_split(features_lst, labels_lst, test_size=0.2, random_state=42)

In [72]:
lda = LinearDiscriminantAnalysis()

# Construct a pipeline with scaling and LDA
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Scale features
    ('lda', lda)
])

# Train the LDA model
pipeline.fit(X_train, y_train)

# Predict on the test set
y_pred = pipeline.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.97810300525081


In [73]:
import joblib
joblib.dump(pipeline, 'lda_apr10.pkl')

['lda_apr10.pkl']

In [3]:
import joblib
pipeline_loaded = joblib.load('model/lda_apr10.pkl')

test_im = cv2.resize(cv2.imread('clipped_roi/3.jpg', cv2.IMREAD_GRAYSCALE), (64, 64))
# test_im = cv2.resize(cv2.imread('window0_40.jpg', cv2.IMREAD_GRAYSCALE), (64, 64))
features = hog(test_im, pixels_per_cell=(4, 4), cells_per_block=(2, 2), transform_sqrt=True, block_norm="L2")
    
features = np.array(features).reshape(1, -1)
predicted_class = pipeline_loaded.predict(features)
with open('data/classes.txt', 'r') as file:
    label = []
    for line in file:
        label.append(line.strip())

print("Predicted Class:", predicted_class)
if predicted_class == 'background':
    predicted_class = -1
else:
    print("Predicted Class:", label[int(predicted_class[0])])
# print(predicted_class)
# print(label[int(predicted_class[0])])


Predicted Class: ['background']
